<a href="https://colab.research.google.com/github/sh20022002/small-Language-Model/blob/main/slm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving tokenizer.zip to tokenizer.zip


In [8]:
import zipfile, os, sys

# Unzip the uploaded file
with zipfile.ZipFile("tokenizer.zip", 'r') as zip_ref:
    zip_ref.extractall("tokenizer")

# Add to sys.path so Python can find your modules
sys.path.append(os.path.abspath("tokenizer"))


In [9]:
!pip install datasets

In [10]:
from datasets import load_dataset
dataset = load_dataset("tatsu-lab/alpaca")


In [11]:
dataset["train"][0]['output']

'1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'

In [13]:
# @markdown Dependency: 5Jwk6myk_LdW
from tokenizer import HybridTokenizer, Transformer, train_model
import torch

In [14]:
device = torch.cuda if torch.cuda.is_available() else "cpu"

In [24]:
!ls tokenizer/tokenizer/


create_t_f.py  __pycache__  scrshots	 tokenizer_state.pkl.gz  transformer.py
__init__.py    README.md    tokenize.py  train.py


sample_data  tokenizer	tokenizer.zip


In [25]:
tok = HybridTokenizer.load('tokenizer/tokenizer/tokenizer_state.pkl.gz')

In [30]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset


class QADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Prepare raw text
        prompt = f"Q: {item['input']}\nA:"
        target = item['output']

        # Encode input and target using your tokenizer
        input_ids = self.tokenizer.encode(prompt)[:self.max_length]
        label_ids = self.tokenizer.encode(target)[:self.max_length]

        # Convert to tensors
        input_tensor = torch.tensor(input_ids, dtype=torch.long)
        label_tensor = torch.tensor(label_ids, dtype=torch.long)

        return {
            "input_ids": input_tensor,
            "labels": label_tensor
        }

def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    labels = [item["labels"] for item in batch]

    padded_inputs = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=0)
    padded_labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

    return {
        "input_ids": padded_inputs,
        "labels": padded_labels
    }



dataset = QADataset(dataset['train'], tok)

train_data, val_data = train_test_split(dataset, test_size=0.1, random_state=42)
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False, collate_fn=collate_fn)


TypeError: Wrong key type: '1557' of type '<class 'numpy.int64'>'. Expected one of int, slice, range, str or Iterable.

In [ ]:
model = Transformer(
    vocab_size=len(tok.token2id),
    dim=512,
    depth=6,
    heads=8,
    mlp_dim=2048,
    window=256
).to(device)
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=3e-4)

train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    epochs=5
)


The `TypeError` in the `train_test_split` function arises because `sklearn`'s `train_test_split` is not designed to directly handle custom PyTorch `Dataset` objects. To resolve this, we will first split the underlying data (the list of dictionaries) into training and validation sets using `train_test_split`, and then create `QADataset` instances for each split.